### Описание

Считываем базу данных приложения по контролю за доходами/расходами и формируем из нее csv-файл для загрузки в новое приложение.

In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect('expense_manager')

In [17]:
sql_query = (f"SELECT * FROM  'xpncetb' ")
expence = pd.read_sql(sql_query, con, index_col='id')
expence

,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location
id,,,,,,,,,,
1,6,,2014-07-23,300,23:53,,0,None,None,None
2,11,,2014-07-23,200,23:55,,0,None,None,None
3,3,Пущино - Серпухов,2014-07-24,62,14:44,,0,None,None,None
4,7,шампунь\nгель для душа\nдве пасты\nдве щетки,2014-07-24,660,16:4,,0,None,None,None
5,2,Pazzo,2014-07-24,1000,17:14,,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2916,6,крупы,2020-03-11,530,16:56,,0,None,None,None
2917,3,москва,2020-03-16,590,16:56,,0,None,None,None
2918,3,москва,2020-03-13,590,16:56,,0,None,None,None


In [62]:
"""
Создаем словарь категорий трат для последующего использования в создании единой таблицы трат
"""

sql_query = (f"SELECT * FROM  'xpncecattb' ")
expence = pd.read_sql(sql_query, con)
expence_category = dict(zip(expence.id, expence.cname))

In [64]:
"""
Создаем словарь категорий приходов для последующего использования в создании единой таблицы приходов
"""

sql_query = (f"SELECT * FROM  'incmcattb' ")
income = pd.read_sql(sql_query, con)
income_category = dict(zip(income.id, income.cname))

In [65]:
income_category

{2: 'traineeship',
 3: 'excursion',
 6: 'salary',
 7: 'advance',
 8: 'salary',
 9: 'grands',
 10: 'different resources',
 11: 'transfer'}

In [66]:
sql_query = (f"SELECT * FROM  xpncetb ")
expence = pd.read_sql(sql_query, con)
expence

,id,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location
0,1,6,,2014-07-23,300,23:53,,0,None,None,None
1,2,11,,2014-07-23,200,23:55,,0,None,None,None
2,3,3,Пущино - Серпухов,2014-07-24,62,14:44,,0,None,None,None
3,4,7,шампунь\nгель для душа\nдве пасты\nдве щетки,2014-07-24,660,16:4,,0,None,None,None
4,5,2,Pazzo,2014-07-24,1000,17:14,,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2912,2916,6,крупы,2020-03-11,530,16:56,,0,None,None,None
2913,2917,3,москва,2020-03-16,590,16:56,,0,None,None,None
2914,2918,3,москва,2020-03-13,590,16:56,,0,None,None,None
2915,2919,3,москва,2020-03-02,240,16:57,,0,None,None,None


In [67]:
sql_query = (f"SELECT * FROM  'incmetb' ")
income = pd.read_sql(sql_query, con)
income

,id,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location
0,1,3,Excursion from “Pansionat“,2014-07-15,3000,14:55,,0,None,None,None
1,2,7,,2014-07-24,3000,17:28,,0,None,None,None
2,3,3,,2014-07-25,1000,14:35,,0,None,None,None
3,4,10,форсайт,2014-08-03,500,12:11,,0,None,None,None
4,5,9,Оклад,2014-08-06,7294,18:26,,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
306,308,8,,2019-12-31,14424,15:20,,0,None,None,None
307,309,7,,2020-01-04,6000,12:44,,0,None,None,None
308,310,8,,2020-01-27,32187,12:44,,0,None,None,None
309,311,7,,2020-02-17,6000,11:9,,0,None,None,None


In [71]:
expence = expence[['cid', 'desc', 'itmdate', 'amnt', 'time']]

In [75]:
expence['category'] = None

c:\users\phoen\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
for idx, row in expence.iterrows():
    expence.loc[idx, 'category'] = expence_category[row.cid]

c:\users\phoen\python\python37\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [78]:
expence

,cid,desc,itmdate,amnt,time,category
0,6,,2014-07-23,300,23:53,food
1,11,,2014-07-23,200,23:55,phone
2,3,Пущино - Серпухов,2014-07-24,62,14:44,travel
3,7,шампунь\nгель для душа\nдве пасты\nдве щетки,2014-07-24,660,16:4,shopping
4,2,Pazzo,2014-07-24,1000,17:14,entertainment
...,...,...,...,...,...,...
2912,6,крупы,2020-03-11,530,16:56,food
2913,3,москва,2020-03-16,590,16:56,travel
2914,3,москва,2020-03-13,590,16:56,travel
2915,3,москва,2020-03-02,240,16:57,travel


In [79]:
income['category'] = None

In [80]:
for idx, row in income.iterrows():
    income.loc[idx, 'category'] = income_category[row.cid]

In [81]:
income

,id,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location,category
0,1,3,Excursion from “Pansionat“,2014-07-15,3000,14:55,,0,None,None,None,excursion
1,2,7,,2014-07-24,3000,17:28,,0,None,None,None,advance
2,3,3,,2014-07-25,1000,14:35,,0,None,None,None,excursion
3,4,10,форсайт,2014-08-03,500,12:11,,0,None,None,None,different resources
4,5,9,Оклад,2014-08-06,7294,18:26,,1,None,None,None,grands
...,...,...,...,...,...,...,...,...,...,...,...,...
306,308,8,,2019-12-31,14424,15:20,,0,None,None,None,salary
307,309,7,,2020-01-04,6000,12:44,,0,None,None,None,advance
308,310,8,,2020-01-27,32187,12:44,,0,None,None,None,salary
309,311,7,,2020-02-17,6000,11:9,,0,None,None,None,advance


Что нужно сделать:

+ Объеденить таблицы
+ проиндексировать по времени добавления
+ отсортировать по дате
+ переопределить категории и подкатегории
+ сохранить в формате, принятом в новом Expence Manager